##  3.2 UM-Bridge Integration for the target function

We give an example code how the posterior density for our model can look like as an UM-Bridge model. We use the log normal distribution since it is more common and for the MCMC algorithm the acceptance probability consists of the substraction instead of the quotient of the target distributions, which makes it more stable.

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
import umbridge
import nest_asyncio
nest_asyncio.apply()

class LogTargetDistributionModel(umbridge.Model):
    def __init__(self):
        super().__init__("posterior")

    def get_input_sizes(self, config):
        return [2] # 2D input

    def get_output_sizes(self, config):
        return [1] # 1D output log density

    def __call__(self, parameters, config):
        x = np.array(parameters[0])
        mean = np.array([0, 0])
        cov = np.array([[1, 0.5], [0.5, 1]])
        posterior = multivariate_normal.logpdf(x, mean, cov)
        return [[posterior]]

    def supports_evaluate(self):
        return True
    
# Create an instance of the model
model = LogTargetDistributionModel()

# Starting the server
umbridge.serve_models([model], 4242)

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)
